In [ ]:
# Remote Training on SSH with CUDA - Hydra Sweep

This notebook lets you launch a **Hydra multirun sweep** on an SSH machine with CUDA and safely disconnect while training continues.

**Sweep Configuration:**
- `model.hidden_channels`: [32, 64, 128, 256, 512]
- `model.conv_types`: 1-layer (GEN, GCN, GAT) and 2-layer combinations

**How it works:**
- Cell 2: Launches sweep as a background process with `nohup` (survives SSH disconnect)
- Cell 3: Monitor recent sweep logs
- Cell 4: Check GPU utilization
- Cell 5: Stop sweep if needed
- Cell 6: View sweep results summary

**Workflow:**
1. Run Cell 2 to start sweep
2. Disconnect from SSH/close notebook - sweep continues!
3. Reconnect later and run Cell 3 to check progress
4. Check results in `multirun/` directory when done

In [9]:
# This is for sweep

import os
from datetime import datetime

# Create logs directory if it doesn't exist
os.makedirs('logs', exist_ok=True)

# Generate unique log filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_file = f"logs/sweep_{timestamp}.log"

print(f"Starting Hydra sweep in background...")
print(f"Logs will be written to: {log_file}")

# Hydra multirun command with sweep parameters
# -m flag enables multirun mode
# Sweep over hidden_channels and conv_types
cmd = (
    f"nohup python main.py -m "
    f"training.loss.adaptive.strategy=fixed,equal_init,equal_init_ema,ema "
    f"training.loss.use_pi=true,false "
    f"training.loss.use_rk4=true,false "
    f"training.loss.use_energy=true,false "
    f"&> {log_file} &"
)

print(f"\nCommand: {cmd}")
os.system(cmd)

print("\n✓ Hydra sweep launched in background!")
print(f"\nUseful commands:")
print(f"  Monitor live:     tail -f {log_file}")
print(f"  Check status:     ps aux | grep 'python main.py'")
print(f"  Kill if needed:   pkill -f 'python main.py'")
print(f"  GPU usage:        nvidia-smi")
print(f"  Results:          ls -lh multirun/")
print(f"\nResults will be saved in: multirun/YYYY-MM-DD/HH-MM-SS/")

Starting Hydra sweep in background...
Logs will be written to: logs/sweep_20251028_142120.log

Command: nohup python main.py -m training.loss.adaptive.strategy=fixed,equal_init,equal_init_ema,ema training.loss.use_pi=true,false training.loss.use_rk4=true,false training.loss.use_energy=true,false &> logs/sweep_20251028_142120.log &

✓ Hydra sweep launched in background!

Useful commands:
  Monitor live:     tail -f logs/sweep_20251028_142120.log
  Check status:     ps aux | grep 'python main.py'
  Kill if needed:   pkill -f 'python main.py'
  GPU usage:        nvidia-smi
  Results:          ls -lh multirun/

Results will be saved in: multirun/YYYY-MM-DD/HH-MM-SS/
[2025-10-28 14:21:25,266][HYDRA] Launching 32 jobs locally
[2025-10-28 14:21:25,266][HYDRA] 	#0 : training.loss.adaptive.strategy=fixed training.loss.use_pi=True training.loss.use_rk4=True training.loss.use_energy=True
[2025-10-28 14:21:25,372][__main__][INFO] - ==================================================
[2025-10-28 14:

In [ ]:
# Monitor the most recent sweep log
import glob
import time

# Find most recent log file
log_files = sorted(glob.glob('logs/sweep_*.log'))
if log_files:
    latest_log = log_files[-1]
    print(f"Monitoring: {latest_log}\n")
    print("="*70)
    
    # Display last 50 lines to see multiple runs
    with open(latest_log, 'r') as f:
        lines = f.readlines()
        for line in lines[-50:]:
            print(line.rstrip())
    
    print("\n" + "="*70)
    print(f"Total lines in log: {len(lines)}")
else:
    print("No sweep logs found yet.")

In [ ]:
# Check GPU utilization
import subprocess

result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print(result.stdout)

In [6]:
import subprocess

print("Stopping all training processes...")
subprocess.run(['pkill', '-f', 'python main.py'])
print("✓ Training processes stopped")

# Verify
result = subprocess.run(['pgrep', '-f', 'python main.py'], capture_output=True)
if result.stdout:
    print("⚠ Some processes still running, try again or use: kill -9 <PID>")
else:
    print("✓ All training processes terminated")

Stopping all training processes...
✓ Training processes stopped
✓ All training processes terminated
